In [1]:
# library required
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

Using ML alogrithum to generate tweet as naive bayes classifer is used for text classification. It predicts the class or category of a given text based on the features extracted from the training data.

Let's load our dataset.

The Dataset have some encoding. So, I have applied encoding parameter so our kernel can read the file.

In [3]:
header = ['Target','Ids','Date','Flag','User','Tweet']
df = pd.read_csv(r"H:\My Drive\Intershala internship\Kudosware\training.1600000.processed.noemoticon.csv",encoding='ISO-8859–1', names=header)

So now, To clean the tweets and to take out useful parameter I used nltk library and some function.

In [3]:

df['Text'] = df['Tweet'].str.lower()
df['Text'] = df['Text'].map(lambda s: ' '.join([x for x in s.split() if 'http' not in x if '@' not in x]))


In [4]:
df['processed_tweets'] = df['Text'].str.replace('[^\w\s]', '').str.lower()


C:\Users\saksh\AppData\Local\Temp\ipykernel_14408\94996927.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['processed_tweets'] = df['Text'].str.replace('[^\w\s]', '').str.lower()


Let's separate each work by using word_tokenizer and also remove stopwords

In [5]:
df['tokenized_tweets'] = df['processed_tweets'].apply(word_tokenize)


In [6]:
stop_words = set(stopwords.words('english'))
df['filtered_tweets'] = df['tokenized_tweets'].apply(lambda x: [word for word in x if word not in stop_words])
# df['filtered_tweets'] = df['filtered_tweets'].map(lambda y: ' '.join([word for word in y]))

As we are training our model on the text so we remove stopwords the model will not learn the right way to write a tweet.

In [7]:
df

,Target,Ids,Date,Flag,User,Tweet,Text,processed_tweets,tokenized_tweets,filtered_tweets
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that's a bummer. you shoulda got david...",awww thats a bummer you shoulda got david car...,"[awww, thats, a, bummer, you, shoulda, got, da...","[awww, thats, bummer, shoulda, got, david, car..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...","[upset, cant, update, facebook, texting, might..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball. managed to sa...,i dived many times for the ball managed to sav...,"[i, dived, many, times, for, the, ball, manage...","[dived, many, times, ball, managed, save, 50, ..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am ...",no its not behaving at all im mad why am i her...,"[no, its, not, behaving, at, all, im, mad, why...","[behaving, im, mad, cant, see]"
...,...,...,...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,just woke up. having no school is the best fee...,just woke up having no school is the best feel...,"[just, woke, up, having, no, school, is, the, ...","[woke, school, best, feeling, ever]"
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdb.com - very cool to hear old walt interv...,thewdbcom very cool to hear old walt intervie...,"[thewdbcom, very, cool, to, hear, old, walt, i...","[thewdbcom, cool, hear, old, walt, interviews, â]"
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,are you ready for your mojo makeover? ask me f...,are you ready for your mojo makeover ask me fo...,"[are, you, ready, for, your, mojo, makeover, a...","[ready, mojo, makeover, ask, details]"
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happy 38th birthday to my boo of alll time!!! ...,happy 38th birthday to my boo of alll time tup...,"[happy, 38th, birthday, to, my, boo, of, alll,...","[happy, 38th, birthday, boo, alll, time, tupac..."


In [8]:
vocab = 0
for token in df['processed_tweets']:
    vocab += len(token)

So to convert a collection of raw text documents into a numerical format i have used TfidfVectorizer as it also have basic pre-processing of nltk which will be help for our model

In [10]:
vector= TfidfVectorizer(max_features=vocab)
train= vector.fit_transform(df['processed_tweets'])

Applied my MultinomialNB model to our data.

In [11]:
MNB = MultinomialNB()
MNB.fit(train,df['Target'])

MultinomialNB()

Ignore the above text.

Created a function to generator text with the help of our model as it generates the next word based on the predicted class by randomly choosing a word from the corresponding subset in the DataFrame.

In [12]:
def text_generator(seed_sentence):
    
    vectorized_sentence = vector.transform([seed_sentence])
    prediction = MNB.predict(vectorized_sentence)[0]
    if prediction == 0:
        next_word = np.random.choice(df[df['Target']==0]['processed_tweets'])
    elif prediction == 2:
        next_word = np.random.choice(df[df['Target']==2]['processed_tweets'])
    else:
        next_word = np.random.choice(df[(df['Target']==4)]['processed_tweets'])
    seed_sentence += " " + next_word
    
    return seed_sentence
        

As I mentioned above that the model randomly choose word so the tweet generatered will hve no sense and we can post it.

In [13]:
seed_sentence = "I have"
generated_text = text_generator(seed_sentence)
print(generated_text)

I have getting ready for work its the last week with my kids and seeing that cuty


In [15]:
seed_sentence = "I love"
generated_text = text_generator(seed_sentence)
print(generated_text)

I love good band


In [16]:
seed_sentence = "World is"
generated_text = text_generator(seed_sentence)
print(generated_text)

World is finally got myself back in the gym today and feeling much more like myself againthank you gb


In [17]:
seed_sentence = "My machine learning"
generated_text = text_generator(seed_sentence)
print(generated_text)

My machine learning drinking a beer and relaxing after riding across kansas on her bike and rehearsing for the show all day


In [18]:
seed_sentence = "I am"
generated_text = text_generator(seed_sentence)
print(generated_text)

I am missing kk already


In [19]:
seed_sentence = "awesome day"
generated_text = text_generator(seed_sentence)
print(generated_text)

awesome day that airport is my daughters namesake


So I have tried a new approach which is LSTM but requires a lot of processing which a TPU can provide.

I have tried implementing it but as i don't have access to TPU so i was not able to tune it or go further.